<a href="https://colab.research.google.com/github/Jarcos09/Tareas/blob/main/docling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from docling.document_converter import DocumentConverter

import pandas as pd
import os

In [ ]:
from huggingface_hub import login

In [ ]:
# --- Configuración ---
# Puedes usar una URL o una ruta a un archivo PDF local.
# Para este ejemplo, usaré un PDF de ejemplo que podrías reemplazar.
# Si tienes un PDF local, asegúrate de que la ruta sea correcta.

# pdf_source = "ruta/a/tu/documento.pdf" # Descomenta y usa tu propio PDF local
pdf_source = os.path.join('Documentos', 'PRUEBA.pdf')

# --- Inicializar el conversor de Docling ---
converter = DocumentConverter()

print(f"Procesando el PDF desde: {pdf_source}")

Procesando el PDF desde: Documentos/PRUEBA.pdf


In [ ]:
try:
    # --- Convertir el documento ---
    # Docling analizará el PDF y extraerá su estructura, texto, tablas, etc.
    result = converter.convert(pdf_source)
    doc = result.document

    print("\n--- Información General del Documento ---")
    dict_repr = doc.export_to_dict()
    num_texts = len(dict_repr.get('texts', []))
    num_tables = len(dict_repr.get('tables', []))

    print(f"Elementos de texto encontrados: {num_texts}")
    print(f"Tablas encontradas: {num_tables}")

    # --- Extraer y mostrar los primeros párrafos ---
    print("\n--- Primeros Párrafos ---")
    # Itera sobre los elementos del documento para encontrar párrafos.
    # Docling organiza el contenido en una estructura jerárquica.
    paragraph_count = 0
    for item, _ in doc.iterate_items():
        if item.label == "paragraph":
            print(f"- {item.text[:200]}...") # Imprime los primeros 200 caracteres de cada párrafo
            paragraph_count += 1
            if paragraph_count >= 5: # Mostrar solo los primeros 5 párrafos como ejemplo
                break
    if paragraph_count == 0:
        print("No se encontraron párrafos.")

    # --- Extraer y mostrar tablas ---
    print("\n--- Tablas Encontradas ---")
    table_count = 0
    if doc.tables:
        for i, table_item in enumerate(doc.tables):
            print(f"\nTabla {i + 1}:")
            try:
                # Intenta exportar la tabla a Markdown o HTML
                # Usaremos to_markdown() para una representación legible
                table_markdown = table_item.to_markdown()
                print(table_markdown)

                # Si quieres intentar convertir a DataFrame de Pandas,
                # a veces es posible si el Markdown es simple o lo conviertes a HTML primero.
                # Para este ejemplo, solo mostraremos el Markdown.
                # Si necesitas el DataFrame, podrías explorar librerías como `markdown_tables` o `tabulate`
                # para parsear el markdown a una estructura que Pandas pueda leer.

            except Exception as e:
                print(f"No se pudo exportar la tabla {i+1} a Markdown: {e}")
            table_count += 1
    if table_count == 0:
        print("No se encontraron tablas.")

    # --- Extracción de encabezados (headings) ---
    print("\n--- Encabezados (Headings) ---")
    heading_count = 0
    for item, _ in doc.iterate_items():
        if item.label == "heading":
            print(f"- Nivel {item.depth}: {item.text}")
            heading_count += 1
            if heading_count >= 5: # Mostrar solo los primeros 5 encabezados como ejemplo
                break
    if heading_count == 0:
        print("No se encontraron encabezados.")


except Exception as e:
    print(f"Ocurrió un error al procesar el documento: {e}")


--- Información General del Documento ---
Elementos de texto encontrados: 21
Tablas encontradas: 1

--- Primeros Párrafos ---
No se encontraron párrafos.

--- Tablas Encontradas ---

Tabla 1:
No se pudo exportar la tabla 1 a Markdown: 'TableItem' object has no attribute 'to_markdown'

--- Encabezados (Headings) ---
No se encontraron encabezados.


In [ ]:
tabla = dict_repr.get('tables')[0]

In [ ]:
for celda in tabla['data']['table_cells']:
  print(celda['text'])

No
Registro Patronal
Razón social
E9211443101
ENERGETICOS GOJISA SA DE C.V
2
F0513478100
MAYA RHI SAUSI RODRIGUEZ
3
F0517821107
PUGA AGUILAR JOSE GPE
F0520223101
MACARIO VELAZQUEZ CASTILLO
F0541768100
SOFTMAT SA DE CV
F0542258101
SUMINISTROS INDUSTRIALES ESPECIALIZADOS VICTORIA SA DE CV
F0542563104
GERARDO RODRIGUEZ LAVIN
8
F0542951101
MIRIAM ELIZABETH RODRIGUEZ OCHOA
9
F0545695101
EDNA KARINA GUZMAN CONDE
10
F0545797105
ROSALBA GUADALUPE BRIONES MENDEZ
11
F0546298103
VANESSA RAQUEL AGUIAR CASTAÑEDA
12
F0548077109
SAUL ECHAVARRI NAVA
13
F0548723108
JV CARTIN SA DE CV
14
Q1811715109
JUVENTINO RUBEN LOZANO HERNANDEZ
15
Q1811758109
PROVEEDORA DE EQUIPO DE LABORATORIO, CURACION Y MEDICAMENTO MANIF SA DECV
16
Q2510390103
RAFAEL DE ALBA VARGAS
17
02511341105
IRENE HERMELINDA CRUZ GUEVARA
18
X4910391104
JULIO CESAR MEDINA HERNANDEZ
19
F0543726106
BRENDA JUDITH LOZANO SOSA


In [ ]:
# 1. Determinar las dimensiones de la tabla
max_row_idx = 0
max_col_idx = 0

for cell in tabla['data']['table_cells']:
    # end_row_offset_idx es el límite exclusivo, así que el índice máximo es este valor - 1.
    # Pero para la dimensión total, el end_row_offset_idx más alto ya nos da el número de filas.
    if cell['end_row_offset_idx'] > max_row_idx:
        max_row_idx = cell['end_row_offset_idx']
    if cell['end_col_offset_idx'] > max_col_idx:
        max_col_idx = cell['end_col_offset_idx']

num_rows = max_row_idx
num_cols = max_col_idx

print(f"La tabla tiene {num_rows} renglones y {num_cols} columnas.")



La tabla tiene 20 renglones y 3 columnas.


In [ ]:
# 2. Inicializar una matriz vacía para la tabla
# Usamos None como valor predeterminado para celdas vacías
table_matrix = [[None for _ in range(num_cols)] for _ in range(num_rows)]

# 3. Llenar la matriz con el texto de las celdas
for cell in tabla['data']['table_cells']:
    row_start = cell['start_row_offset_idx']
    col_start = cell['start_col_offset_idx']
    row_span = cell['row_span']
    col_span = cell['col_span']
    text = cell['text']

    # Asignar el texto a la posición correcta, considerando row_span y col_span
    # Aunque en tus datos todos son span 1, este código es más robusto.
    for r_offset in range(row_span):
        for c_offset in range(col_span):
            current_row = row_start + r_offset
            current_col = col_start + c_offset
            if 0 <= current_row < num_rows and 0 <= current_col < num_cols:
                table_matrix[current_row][current_col] = text
            else:
                print(f"Advertencia: Celda fuera de límites calculados: {cell}")

# 4. Identificar los nombres de las columnas (headers)
column_names = [None] * num_cols
for cell in tabla['data']['table_cells']:
    if cell['column_header']:
        # Asume que los encabezados están en el primer row_offset_idx de los headers
        if cell['start_row_offset_idx'] == 0:
            col_idx = cell['start_col_offset_idx']
            if 0 <= col_idx < num_cols:
                column_names[col_idx] = cell['text']

# Si algunos nombres de columna no se encontraron o son None (por ejemplo, si la primera fila no tuviera encabezados para todas las columnas), puedes usar nombres genéricos
for i, name in enumerate(column_names):
    if name is None:
        column_names[i] = f'Columna_{i}'

# 5. Crear el DataFrame de pandas
# Eliminar la fila de encabezados de la matriz si los encabezados se usarán como nombres de columna
# En tu caso, los encabezados están en la fila 0 (start_row_offset_idx: 0)
# Así que los datos "reales" de la tabla empiezan desde la fila 1.
data_rows = table_matrix[1:] # Excluye la primera fila que son los headers

df = pd.DataFrame(data_rows, columns=column_names)

# Opcional: Si la primera columna 'No' es un índice y no un dato, puedes establecerla como índice
# Si es un dato que quieres conservar, simplemente omite esta línea.
if 'No' in df.columns:
    df = df.set_index('No')

# Mostrar el DataFrame
print("\nDataFrame generado:")
print(df)

df.to_csv("prueba.csv", index=True, encoding="latin1")




DataFrame generado:
     Registro Patronal                                       Razón social
No                                                                       
None       E9211443101                       ENERGETICOS GOJISA SA DE C.V
2          F0513478100                           MAYA RHI SAUSI RODRIGUEZ
3          F0517821107                              PUGA AGUILAR JOSE GPE
None       F0520223101                         MACARIO VELAZQUEZ CASTILLO
None       F0541768100                                   SOFTMAT SA DE CV
None       F0542258101  SUMINISTROS INDUSTRIALES ESPECIALIZADOS VICTOR...
None       F0542563104                            GERARDO RODRIGUEZ LAVIN
8          F0542951101                   MIRIAM ELIZABETH RODRIGUEZ OCHOA
9          F0545695101                           EDNA KARINA GUZMAN CONDE
10         F0545797105                   ROSALBA GUADALUPE BRIONES MENDEZ
11         F0546298103                    VANESSA RAQUEL AGUIAR CASTAÑEDA
12         F05480

In [ ]:
dict_repr['texts'][21]

IndexError: list index out of range